我们将使用GroupLens研究小组收集的MovieLens数据集。这个数据集描述了MovieLens的五星评级和标记活动。该数据集包含来自600多名用户的9000多部电影的约10万个评分。我们将使用该数据集生成两种节点类型，分别保存电影和用户的数据，以及一种连接用户和电影的边类型，表示用户对特定电影的评分关系。

首先，我们将数据集下载到任意文件夹（在本例中为当前目录）：

In [1]:
from torch_geometric.data import download_url, extract_zip

url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, './data'), './data')

movies_path = './data/ml-latest-small/movies.csv'
ratings_path = './data/ml-latest-small/ratings.csv'

Using existing file ml-latest-small.zip
Extracting ./data\ml-latest-small.zip


In [2]:
import pandas as pd

print(len(pd.read_csv(movies_path)))
pd.read_csv(movies_path).head(10)

9742


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [3]:
print(len(pd.read_csv(ratings_path)))
pd.read_csv(ratings_path).head(10)

100836


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


为了用PyG数据格式表示这些数据，我们首先定义了一个方法load_node_csv()，该方法读取*.csv文件并返回形状为[num_nodes，num_features]的节点级特征表示x：

In [4]:
import torch

def load_node_csv(path, index_col, encoders=None, **kwargs): # **kwargs用于在函数定义中接收任意数量的关键字参数，是一个字典
    df = pd.read_csv(path, index_col=index_col, **kwargs) # 读取*.csv
    mapping = {index: i for i, index in enumerate(df.index.unique())} # 将索引映射成连续值

    x = None
    if encoders is not None:
        xs = [encoder(df[col]) for col, encoder in encoders.items()]
        x = torch.cat(xs, dim=-1)

    return x, mapping

这里，load_node_csv()从路径读取*.csv文件，并创建一个字典映射，将其索引列映射到范围｛0，…，num_rows-1｝中的连续值。这是必要的，因为我们希望我们的最终数据表示尽可能紧凑，例如，第一行中的电影表示应该可以通过x[0]访问。

In [5]:
from sentence_transformers import SentenceTransformer
class SequenceEncoder:
    def __init__(self, model_name='all-MiniLM-L6-v2', device=None):
        self.device = device
        self.model = SentenceTransformer(model_name, device=device)

    @torch.no_grad()
    def __call__(self, df):
        x = self.model.encode(df.values, show_progress_bar=True,
                              convert_to_tensor=True, device=self.device)
        print(x.shape)
        return x.cpu()

SequenceEncoder类加载一个由model_name给定的预先训练的NLP模型，并使用它将字符串列表编码为形状为[num_strings,embedding_dim]的PyTorch张量。我们可以使用此SequenceEncoder对movies.csv文件的标题进行编码。

以类似的方式，我们可以创建另一个编码器，将电影类型转换为分类标签。为此，我们首先需要找到数据中存在的所有电影类型，创建shape[num_movies，num_genres]的特征表示x，并在类型j存在于电影i中的情况下将1分配给x[i，j]：

In [6]:
class GenresEncoder:
    def __init__(self, sep='|'):
        self.sep = sep

    def __call__(self, df):
        genres = set(g for col in df.values for g in col.split(self.sep))
        mapping = {genre: i for i, genre in enumerate(genres)}

        x = torch.zeros(len(df), len(mapping))
        for i, col in enumerate(df.values):
            for genre in col.split(self.sep):
                x[i, mapping[genre]] = 1
                
        print(x.shape)
        return x

有了这个，我们可以通过以下方式获得我们对电影的最终呈现：

In [7]:
movie_x, movie_mapping = load_node_csv(
    movies_path, index_col='movieId', encoders={
        'title': SequenceEncoder(),
        'genres': GenresEncoder()
    })

print(movie_x)

Batches:   0%|          | 0/305 [00:00<?, ?it/s]

torch.Size([9742, 384])
torch.Size([9742, 20])
tensor([[-0.0828,  0.0530,  0.0536,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1053,  0.1508, -0.0264,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0988,  0.0176, -0.0527,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.1115,  0.0310, -0.0177,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0366,  0.0137,  0.0315,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0500, -0.0141, -0.0031,  ...,  0.0000,  0.0000,  0.0000]])


类似地，我们也可以使用load_node_csv()来获得从userId到连续值的用户映射。但是，此数据集中没有用户的其他特征信息。因此，我们没有定义任何编码器：

In [8]:
_, user_mapping = load_node_csv(ratings_path, index_col='userId')
print(user_mapping)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 119, 121: 120, 122: 12

这样，我们就可以初始化HeteroData对象，并将两种节点类型传递给它：

In [9]:
from torch_geometric.data import HeteroData

data = HeteroData()

data['user'].num_nodes = len(user_mapping)  # "user"没有任何特征
data['movie'].x = movie_x

print(data)
print(movie_x.shape)

HeteroData(
  user={ num_nodes=610 },
  movie={ x=[9742, 404] }
)
torch.Size([9742, 404])


由于用户没有任何节点级别的信息，我们只定义其节点数。因此，在异构图模型的训练过程中，我们可能需要通过torch.nn.Embedding以端到端的方式学习不同的用户嵌入。

接下来，我们来看看根据用户的评分将他们与电影联系起来。为此，我们定义了一个方法load_edge_csv()，该方法从ratings.csv返回shape[2，num_ratings]的最终edge_index表示，以及原始*.csv文件中存在的任何其他功能：

In [10]:
def load_edge_csv(path, src_index_col, src_mapping, dst_index_col, dst_mapping,
                  encoders=None, **kwargs):
    df = pd.read_csv(path, **kwargs)

    src = [src_mapping[index] for index in df[src_index_col]]
    dst = [dst_mapping[index] for index in df[dst_index_col]]
    #print(len(src))
    #print(len(dst))
    edge_index = torch.tensor([src, dst])

    edge_attr = None
    if encoders is not None:
        edge_attrs = [encoder(df[col]) for col, encoder in encoders.items()]
        edge_attr = torch.cat(edge_attrs, dim=-1)
        
    #print(edge_attr.shape)

    return edge_index, edge_attr

这里，src_index_col和dst_index_col分别定义源节点和目标节点的索引列。我们进一步利用节点级映射src_mapping和dst_mapping来确保原始索引在我们的最终表示中被映射到正确的连续索引。

对于文件中定义的每条边，它会在src_mapping和dst_mapping中查找正向索引，并适当地移动数据。

与load_node_csv()类似，编码器用于返回额外的边特征信息。例如，为了从ratings.csv中的rating列加载ratings，我们可以定义一个IdentityEncoder，它只需将浮点值列表转换为PyTorch张量：

In [11]:
class IdentityEncoder:
    def __init__(self, dtype=None):
        self.dtype = dtype

    def __call__(self, df):
        return torch.from_numpy(df.values).view(-1, 1).to(self.dtype)

这样，我们就可以完成我们的HeteroData对象了：

In [12]:
edge_index, edge_label = load_edge_csv(
    ratings_path,
    src_index_col='userId',
    src_mapping=user_mapping,
    dst_index_col='movieId',
    dst_mapping=movie_mapping,
    encoders={'rating': IdentityEncoder(dtype=torch.long)},
)

data['user', 'rates', 'movie'].edge_index = edge_index
data['user', 'rates', 'movie'].edge_label = edge_label

print(data)

HeteroData(
  user={ num_nodes=610 },
  movie={ x=[9742, 404] },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836, 1]
  }
)


该HeteroData对象是PyG中异构图的原生格式，可以用作异构图模型的输入。